<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/stormglass_io(10request_per_day).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install openmeteo-requests requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.3/684.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.6 MB/s eta 0:00:00


In [5]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

cities = {
    "Turku": {"lat": 60.45, "lon": 22.26},
    "Copenhagen": {"lat": 55.67, "lon": 12.56},
    "Stockholm": {"lat": 59.32, "lon": 18.06},
    "Oslo": {"lat": 59.91, "lon": 10.75}
}

def fetch_2024_data(name, lat, lon):
    print(f"🌍 Extracting 2024 data for {name}...")
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": "2024-01-01",
        "end_date": "2024-12-31",
        "hourly": ["temperature_2m", "wind_speed_10m", "shortwave_radiation"],
        "wind_speed_unit": "ms",
        "timezone": "UTC"
    }

    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]

    # Process hourly data
    hourly = response.Hourly()
    hourly_data = {
        "timestamp": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temp_c": hourly.Variables(0).ValuesAsNumpy(),
        "wind_ms": hourly.Variables(1).ValuesAsNumpy(),
        "ghi_wm2": hourly.Variables(2).ValuesAsNumpy()
    }

    df = pd.DataFrame(data=hourly_data)
    filename = f"{name.lower()}_weather_2024.csv"
    df.to_csv(filename, index=False)
    print(f"✅ Success! Saved {len(df)} hours to {filename}")

if __name__ == "__main__":
    for city, coords in cities.items():
        fetch_2024_data(city, coords['lat'], coords['lon'])

🌍 Extracting 2024 data for Turku...
✅ Success! Saved 8784 hours to turku_weather_2024.csv
🌍 Extracting 2024 data for Copenhagen...
✅ Success! Saved 8784 hours to copenhagen_weather_2024.csv
🌍 Extracting 2024 data for Stockholm...
✅ Success! Saved 8784 hours to stockholm_weather_2024.csv
🌍 Extracting 2024 data for Oslo...
✅ Success! Saved 8784 hours to oslo_weather_2024.csv
